# LAMMPS Input Files

## Simulation and File Parameters

1. **First Pick potential**
2. **Then pick supercell size/number of atoms**
3. **Then vary simulation parameters:**
   - Composition
   - Temperature
   - Shear strain rate

In [ ]:
from pymatgen.core import Structure, Lattice, Element
from pymatgen.io.lammps.data import LammpsData
import numpy as np
import os

# 1. Select potential
#potential_folder = "2025_Sharifi"
potential_folder = "2022_Mahata"
#potential_folder = "Zhou04_eam_alloy"
#potential_folder = "Cu1_eam_fs"

#potential_files = ["library.meam", "CuAl.meam"]
potential_files = ["library.AlCu.meam", "AlCu.meam"]
#potential_files = ["CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy"]
#potential_files = ["Cu1.eam.fs"]

# 2. Pick supercell size/number of atoms
supercell_size = (14, 14, 14)
data_file = "Al1-xCux.data"

# 3. Pick simulation parameters: composition, temperature and shear strain rate
x = 0.00  # Cu fraction
T = 1500  # Temperature (K)
xyrate_1_per_s = 1e10  # Shear rate (1/s)
xyrate_str = f"{xyrate_1_per_s:.0e}".replace("+0", "").replace("+", "") # For path

# Format supercell_size as a string, e.g., "14x14x14"
supercell_str = f"{supercell_size[0]}x{supercell_size[1]}x{supercell_size[2]}"

# Run number
run_num = 3
# Directory and file path formatting with supercell_size included
path = os.path.join(
    "/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux",
    potential_folder, supercell_str, f"x_{x:.2f}/{T}K/{xyrate_str}/run_{run_num}"
)
os.makedirs(path, exist_ok=True)

## Helper Functions

In [142]:
# Generate Al1-xCux supercell with random Cu substitutions
def generate_al1xcux_supercell(x, supercell_size, lattice_constant):
    lattice = Lattice.cubic(lattice_constant)
    structure = Structure.from_spacegroup(
        "Fm-3m", lattice, ["Al"], [[0, 0, 0]]
    )
    supercell = structure * supercell_size
    num_sites = len(supercell)
    num_cu = int(round(x * num_sites))
    al_indices = [i for i, site in enumerate(supercell) if site.specie == Element("Al")]
    cu_indices = np.random.choice(al_indices, num_cu, replace=False)
    for idx in cu_indices:
        supercell[int(idx)] = "Cu"
    return supercell

# Convenience function to get atom types to write in LAMMPS input script
def get_atom_types(x):
    if x == 0:
        return "Al"
    elif x == 1:
        return "Cu"
    else:
        return "Al Cu"

## Generate and Write LAMMPS Data File

In [143]:
# 2025 Sharifi lattice constants
#Al_lattice_constant = 4.05000  # Angstrom
#Cu_lattice_constant = 3.62000  # Angstrom

# 2022 Mahata lattice constants
Al_lattice_constant = 4.0502  # Angstrom
Cu_lattice_constant = 3.6150  # Angstrom

# Pick the lattice constant based on Vegard's law (linear interpolation)
lattice_constant = Al_lattice_constant * (1 - x) + Cu_lattice_constant * x  

# Generate the supercell and write LAMMPS data file
supercell = generate_al1xcux_supercell(x, supercell_size, lattice_constant)
lammps_data = LammpsData.from_structure(supercell, atom_style="atomic")
lammps_data.write_file(os.path.join(path, data_file))

## Shear Strain Calculation and Simulation Time

In [144]:
# Shear Strain and Simulation Time Calculation
#
# T(t) = T0 + L0 * erate * dt
#
# Where:
#   T(t)   : Tilt factor at time t (same as delta_x)
#   T0     : Initial tilt factor
#   L0     : Original length of the box perpendicular to the shear direction
#   erate  : Shear strain rate
#   dt     : Elapsed time
#
# Example:
#   If erate = 0.01 (1/ps), then:
#     - Shear strain after 1 ps is 0.01
#     - Shear strain after 2 ps is 0.02, etc. 
# This block calculates the total simulation time and total shear strain for a given number of steps and timestep.
# You can use the results here to set step2_run_time and step3_run_time in the next block.

# Parameters
xyrate_1_per_ps = xyrate_1_per_s / 1e12  # Shear rate in 1/ps

dt = 0.001  # Timestep in ps (1 fs)

step2_L0 = 1  # Shear strain for step 2
step3_L0 = 5 # Shear strain for step 3

step2_run_time = int(step2_L0 / xyrate_1_per_ps * 1000)  # Step 2: NEMD equilibration (timesteps yielding step2_L0)
step3_run_time = int(step3_L0 / xyrate_1_per_ps * 1000)  # Step 3: Data gathering (timesteps yielding step3_L0)

## Generate and Write LAMMPS Input Script

In [145]:
atom_types = get_atom_types(x)

# Potential settings
# For 2022_Mahata and 2025_Sharifi
pair_coeff = "meam"

# For 2004 Zhou
#pair_coeff = "eam/alloy"

# For Cu eam
#pair_coeff = "eam/fs"

if pair_coeff == "meam":
    elements = ["Al", "Cu"]
    potential_files_str = f"{potential_files[0]} {' '.join(elements)} {potential_files[1]}"
else:
    potential_files_str = pair_coeff

temp_seed = np.random.randint(100_000, 5_000_000)
step1_run_time = 1_000_000 # Step 1: Temperature equilibration (1 ns)
num_bins = 28 # Number of bins for velocity profile along y direction TODO: consider adjusting this

in_nemd_script = f"""# Sample LAMMPS input script for viscosity of FCC solid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable        P equal 1 
variable        T equal {T}
variable        xyrate_1_per_ps equal {xyrate_1_per_ps}

# Problem setup
units		    metal
dimension	    3
boundary        p p p
atom_style	    atomic

read_data	    Al1-xCux.data
change_box      all triclinic

pair_style      {pair_coeff}
pair_coeff      * * {potential_files_str} {atom_types}

# --- Step 1. Temperature equilibration ---
velocity        all create $T {temp_seed} 
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)

thermo          1000
#dump            equilibration all custom 50 dump_temp_equil id type x y z vx vy vz 
run	            {step1_run_time}

#undump         equilibration
unfix  	        1
reset_timestep  0

# --- Step 2. Turn on NEMD shear and equilibrate some more ---
fix		        1 all nvt/sllod temp $T $T $(100.0*dt)

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${{xyrate_1_per_ps}} remap v

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

#dump            nemd_equilibration all custom 50 dump_nemd_equil id type x y z vx vy vz
run		        {step2_run_time}
#undump          nemd_equilibration
reset_timestep  0

# --- Step 3. Data gathering run ---
# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center {1/num_bins} units reduced
fix		        3 all ave/chunk 10 {int(step3_run_time/10)} {step3_run_time} layers_center vx file vx_profile_run_avg.txt

# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_xyrate_1_per_ps)*0.0001 # Computed every timestep. Convert bar.ps to mPa.s 

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average averages the average values computed every 1000 timesteps.
fix             vprint all print 10 \"${{timestep}} ${{visc}}\" file visc_output.txt screen no
fix             pprint all print 10 \"${{timestep}} ${{neg_pxy}}\" file pxy_output.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 1000 file visc_run_avg.txt

# Calculate the running average of -pxy as well
variable        neg_pxy equal -pxy
fix             pave all ave/time 10 100 1000 v_neg_pxy ave running start 1000 file pxy_run_avg.txt

thermo          1000
thermo_style	custom step temp etotal press pxy ly v_visc f_vave
thermo_modify	temp tilt

dump	        data_gathering all custom {int(step3_run_time/1000)} dump_nemd_analyze id type x y z vx vy vz 

run		        {step3_run_time}
"""

with open(os.path.join(path, "in.nemd"), "w") as f:
    f.write(in_nemd_script)

## Copy Potential File

In [146]:
import shutil
for potential in potential_files:
    src_potential = f"/ocean/projects/dmr190011p/nhew/lammps-workflows/workflows/viscosity/Al1-xCux/potential_files/{potential_folder}/{potential}"
    dst_potential = os.path.join(path, potential)
    shutil.copy(src_potential, dst_potential)

## Write and Submit Job Script

In [147]:
job_script = """#!/bin/bash
#SBATCH --job-name=Bridges-2
#SBATCH -A dmr190011p
#SBATCH -p RM
#SBATCH -N 4
#SBATCH --ntasks-per-node 64
#SBATCH -t 48:00:00

module purge
module load intelmpi/2021.3.0-intel2021.3.0
module load gcc/10.2.0
module load cuda/11.7.1
module load python

echo "SLURM_NTASKS: " $SLURM_NTASKS

ulimit -s unlimited
export OMP_NUM_THREADS=1

mpirun -n $SLURM_NTASKS /opt/packages/LAMMPS/lammps-23Jun2022/build/lmp -sf omp -pk omp $OMP_NUM_THREADS -in in.nemd
"""

with open(os.path.join(path, "job.sh"), "w") as f:
    f.write(job_script)

import subprocess
cwd = os.getcwd()
os.chdir(path)
try:
    subprocess.run(["sbatch", "job.sh"], check=True)
finally:
    os.chdir(cwd)

Submitted batch job 36563496
